# Arabic Poems Generator - NLP Project. 
The Arabic Poems Generator is an NLP project designed to generate Arabic poems.

In [ ]:
!pip install transformers

In [ ]:
!pip install pyarabic

In [ ]:
!pip install gpt_2_simple

In [83]:
import pandas as pd
import numpy as np
from pyarabic.araby import strip_tashkeel, normalize_hamza, tokenize
import gensim.downloader as api
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from pyarabic.araby import strip_harakat
import nltk

In [84]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


## Reading data
The training data for the Arabic Poems Generator is "Arabic Poetry Dataset" from Kaggle, which contains a collection of Arabic poems. I extracted the "poem_text" column for further processing.

In [85]:
ar_poems = pd.read_csv("/content/drive/MyDrive/all_poems.csv")
ar_poems.head()

,poem_id,poem_link,poem_style,poem_text,poem_title,poet_cat,poet_id,poet_link,poet_name
0,21,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,أنشودة المطر,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
1,65546,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,أقداح و أحلام,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
2,65561,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...,هوى واحد !,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
3,65554,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,اساطير من حشرجات الزمان نسيج اليد البالية رواه...,أساطير,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب
4,65550,http://www.adab.com/modules.php?name=Sh3er&doW...,فصحى,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...,اللقاء الأخير,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doW...,بدر شاكر السياب


## General Understanding of the data

In [53]:
print("\nDataframe dimensions:")
print(ar_poems.shape)

print("\nMissing values:")
print(ar_poems.isnull().sum())

print("\nSummary statistics:")
print(ar_poems.describe())


Dataframe dimensions:
(58021, 9)

Missing values:
poem_id       0
poem_link     0
poem_style    0
poem_text     1
poem_title    0
poet_cat      0
poet_id       0
poet_link     0
poet_name     0
dtype: int64

Summary statistics:
            poem_id       poet_id
count  58021.000000  58021.000000
mean   45930.150652    292.302270
std    28698.280675    168.633587
min       10.000000      2.000000
25%    16626.000000    173.000000
50%    56669.000000    246.000000
75%    71653.000000    422.000000
max    86650.000000    755.000000


In [54]:
print("\nPoem Style: ")
ar_poems['poem_style'].value_counts()


Poem Style: 


فصحى    57894
عامي       94
-          33
Name: poem_style, dtype: int64

In [55]:
print("\nPoem origin: ")
ar_poems['poet_cat'].value_counts()


Poem origin: 


العصر العباسي          19417
العصر الأندلسي          6296
سوريا                   5036
لبنان                   4800
مصر                     4793
العراق                  4252
العصر الإسلامي          3700
السعودية                1931
فلسطين                  1713
العصر الجاهلي           1461
اليمن                    700
عمان                     619
الإمارات                 563
البحرين                  555
الجزائر                  405
السودان                  403
تونس                     351
الأردن                   265
ليبيا                    240
المغرب                   202
إيران                     90
أفغانستان                 70
شعراء العراق والشام       68
الكويت                    54
قطر                       22
موريتانيا                 15
Name: poet_cat, dtype: int64

The general understanding of data could help us in future implementations (to perform features engineering). <br>
For now we want the Poem itself, from the poem_text column. 

In [86]:
ar_df = ar_poems[['poem_text']].copy()
ar_df.columns = ['poem']
ar_df["poem"] = ar_df["poem"].astype("str")
ar_df.head()

,poem
0,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...
1,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...
2,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...
3,اساطير من حشرجات الزمان نسيج اليد البالية رواه...
4,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...


In [57]:
# General look at some poems
print("Poem #1:\n", ar_df["poem"][0], "\nNumber of words in the poem:", len(ar_df["poem"][0].split()))
print("Poem #10:\n", ar_df["poem"][10], "\nNumber of words in the poem:", len(ar_df["poem"][10].split()))
print("Poem #100 :\n", ar_df["poem"][100], "\nNumber of words in the poem:", len(ar_df["poem"][100].split()))
print("Poem #1000 :\n", ar_df["poem"][1000], "\nNumber of words in the poem:", len(ar_df["poem"][1000].split()))


Poem #1:
 عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي عنهما القمر عيناك حين تبسمان تورق الكروم وترقص الاضواء كالاقمار في نهر يرجه المجذاف وهنا ساعة السحر كانما تنبض في غوريهما النجوم وتغرقان في ضباب من اسي شفيف كالبحر سرح اليدين فوقه المساء دفء الشتاء فيه وارتعاشة الخريف والموت والميلاد والظلام والضياء فتستفيق ملء روحي رعشة البكاء ونشوة وحشية تعانق السماء كنشوة الطفل اذا خاف من القمر كان اقواس السحاب تشرب الغيوم وقطرة فقطرة تذوب في المطر وكركر الاطفال في عراءش الكروم ودغدغت صمت العصافير علي الشجر انشودة المطر مطر مطر مطر تثاءب المساء والغيوم ما تزال تسح ما تسح من دموعها الثقال كان طفلا بات يهذي قبل ان ينام بان امه – التي افاق منذ عام فلم يجدها ثم حين لج في السءال قالوا له "بعد غد تعود " لا بد ان تعود وان تهامس الرفاق انها هناك في جانب التل تنام نومة الحود تسف من ترابها وتشرب المطر كان صيادا حزينا يجمع الشباك ويلعن المياه والقدر وينثر الغناء حيث يافل القمر مطر مطر اتعلمين اي حزن يبعث المطر وكيف تنشج المزاريب اذا انهمر وكيف يشعر الوحيد فيه بالضياع بلا انتهاء – كالدم المراق كالجياع كال

In [58]:
def has_diacritics(text):
    # Strip tashkeel/diacritics from the text
    stripped_text = strip_tashkeel(text)

    # Check if the stripped text is different from the original text
    return stripped_text != text

has_diacritics_result = ar_df["poem"].apply(has_diacritics)

has_diacritics_result.any()

False

## Preprocessing:
The preprocessing step includes: <br> 
* Stripping diacritics (Tashkeel) 
using the strip_tashkeel function. <br>
PS: although the given data has no diacritics I will consider it as a preprocessing step.
* Normalizing Hamza 
using the normalize_hamza function from the pyarabic.araby library.
* Removing Special Characters 
using regex
* Removing multiple spaces
using regex

In [87]:
import re

ARABIC_PUNCTUATION = '''`÷×؛<>_()&^%][ـ،/:؟.,*'{}~¦+|!""”…“–ـ'''

def remove_arabic_punctuation(text):
    cleaned_text = re.sub('[' + re.escape(ARABIC_PUNCTUATION) + ']', '', text)
    return cleaned_text

def remove_multiple_spaces(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text

text_stripped_tashkeel = ar_df["poem"].apply(strip_tashkeel)
text_normalized_hamza = text_stripped_tashkeel.apply(normalize_hamza)
text_cleaned_punctuation = text_normalized_hamza.apply(remove_arabic_punctuation)
text_cleaned = text_cleaned_punctuation.apply(remove_multiple_spaces)
ar_df["poem"] = text_cleaned

In [60]:
# General look at some poems after preprocessing
print("Poem #1:\n", ar_df["poem"][0], "\nNumber of words in the poem:", len(ar_df["poem"][0].split()))
print("Poem #10:\n", ar_df["poem"][10], "\nNumber of words in the poem:", len(ar_df["poem"][10].split()))
print("Poem #100 :\n", ar_df["poem"][100], "\nNumber of words in the poem:", len(ar_df["poem"][100].split()))
print("Poem #1000 :\n", ar_df["poem"][1000], "\nNumber of words in the poem:", len(ar_df["poem"][1000].split()))

Poem #1:
 عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي عنهما القمر عيناك حين تبسمان تورق الكروم وترقص الاضواء كالاقمار في نهر يرجه المجذاف وهنا ساعة السحر كانما تنبض في غوريهما النجوم وتغرقان في ضباب من اسي شفيف كالبحر سرح اليدين فوقه المساء دفء الشتاء فيه وارتعاشة الخريف والموت والميلاد والظلام والضياء فتستفيق ملء روحي رعشة البكاء ونشوة وحشية تعانق السماء كنشوة الطفل اذا خاف من القمر كان اقواس السحاب تشرب الغيوم وقطرة فقطرة تذوب في المطر وكركر الاطفال في عراءش الكروم ودغدغت صمت العصافير علي الشجر انشودة المطر مطر مطر مطر تثاءب المساء والغيوم ما تزال تسح ما تسح من دموعها الثقال كان طفلا بات يهذي قبل ان ينام بان امه التي افاق منذ عام فلم يجدها ثم حين لج في السءال قالوا له بعد غد تعود لا بد ان تعود وان تهامس الرفاق انها هناك في جانب التل تنام نومة الحود تسف من ترابها وتشرب المطر كان صيادا حزينا يجمع الشباك ويلعن المياه والقدر وينثر الغناء حيث يافل القمر مطر مطر اتعلمين اي حزن يبعث المطر وكيف تنشج المزاريب اذا انهمر وكيف يشعر الوحيد فيه بالضياع بلا انتهاء كالدم المراق كالجياع كالحب كالا

* Splitting the data into train and eval datasets 

In [88]:
train_data = ar_df["poem"][:4000]
train_data

0       عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...
1        انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...
2        علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...
3       اساطير من حشرجات الزمان نسيج اليد البالية رواه...
4       والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...
                              ...                        
3995    حين شهدت ولادتها قلت سيكون لها مستقبل مستقبل م...
3996    في وجهي نظر العراف واغمض عينا واحدة وتطلع نحوي...
3997    شباكها تحتي مباشرة وغرفة نومها مكشوفة وكتابها ...
3998    لم تنجح مريم في التعليم لم تقن قطف الزيتون او ...
3999    فيه النفح الطيب فيه الحب شهم وجواد يدرك ان اله...
Name: poem, Length: 4000, dtype: object

In [89]:
eval_data = ar_df["poem"][4000:6000]
eval_data

4000    حقاني حتي الكفر ومريض بالخير يدعو دوما لايمان ...
4001    اسمي اسمي عكاش العربي عكاش بيتي بيتي في اخر من...
4002    سيظل يصر الشباك حتي تخلعه الريح ويظل يءن الكلب...
4003    لا تحسدوا الشجر الطالع في الفراغ لا تحسدوا الف...
4004    تاكسي المكتب الذي اقلك يستحق العنة ساءق التاكس...
                              ...                        
5995     جر المخزيات علي كليب جرير ثم ما منع الذمارا و...
5996     يا ابن المراغة انما جاريتني بمسبقين لدي الفعا...
5997     عرفت باعلي راءسبعدما مضت سنة ايامها وشهورها م...
5998     اني لينفعني باسي فيصرفني اذا اتي دون شيء مرة ...
5999     اني كتبت اليك التمس الغني بيديك او بيدي ابيك ...
Name: poem, Length: 2000, dtype: object

In [90]:
train_data.to_csv("/content/drive/MyDrive/arabic_train_data.txt",header=None, index=None, sep=' ', mode='a')
eval_data.to_csv("/content/drive/MyDrive/arabic_eval_data.txt",header=None, index=None, sep=' ', mode='a')

## Generating the Model
I'll use GPT2 (Generative Pre-trained Transformer 2) architecture.


In [91]:
import gpt_2_simple as gpt2

In [92]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 308Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.63Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 142Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:13, 37.5Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 338Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 5.31Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.18Mit/s]


In [93]:
file_name = "/content/drive/MyDrive/arabic_train_data.txt"

### Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

<br><br>
Parameters:

* sess: The TensorFlow session in which the fine-tuning will take place.

* dataset: The name or path of the dataset file used for fine-tuning. It should be a plain text file where each training example is on a separate line.

* model_name: The model architecture to use for fine-tuning. It refers to the GPT-2 model variant, such as '124M', '355M', etc. The number represents the number of parameters in millions.

* steps: The number of training steps (iterations) to perform during fine-tuning.

* restore_from: The checkpoint from which to restore the model weights. It can take the following values:
    * 'fresh': Initialize the model weights randomly (starts training from scratch).
    * 'latest': Resume training from the latest checkpoint (continues training from the last saved checkpoint).
    * 'specific': Restore from a specific checkpoint by providing the path or name of the checkpoint file.
* run_name: The name of the run or experiment. It is used to identify and save the checkpoints and training logs associated with this specific run.

* print_every: The frequency (in steps) at which to print the training progress and loss during fine-tuning.

* sample_every: The frequency (in steps) at which to generate sample outputs from the model during fine-tuning. This can be useful to monitor the model's progress and generate creative text samples.

* save_every: The frequency (in steps) at which to save the model checkpoint during fine-tuning. It determines how often the model's weights and optimizer state are saved for future use or evaluation.

In [94]:
sess = gpt2.start_tf_sess()

In [76]:
gpt2.reset_session(sess)

In [31]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=100,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=50,
              save_every=10
              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:15<00:00, 15.88s/it]


dataset has 3709643 tokens
Training...
[10 | 1577.04] loss=2.27 avg=2.27
Saving checkpoint/run1/model-10
[20 | 3090.18] loss=2.20 avg=2.23
Saving checkpoint/run1/model-20


Instructions for updating:
Use standard file APIs to delete files with this prefix.


[30 | 4630.12] loss=2.08 avg=2.18
Saving checkpoint/run1/model-30
[40 | 6189.99] loss=2.23 avg=2.19
Saving checkpoint/run1/model-40
[50 | 7702.72] loss=2.16 avg=2.19
Saving checkpoint/run1/model-50
======== SAMPLE 1 ========
يينا لا تلشكنا وجدتين كثنا علينا ان الوساحن غاراتني الاحمين وظياءكني بيننا اسوقيطني من سموقي علي بالثلعا منا فيفة غيراتقتني لماب الوسوقي بشهار ووحينا عن المطنباتاب بنا فيفة فيفر الراين يشحرني من الدينء رسالكان ولينس ولينس احاكاني حيخري خير علي المسلمل حسين بجد قلب ولريس قلءهني كي اويعد الذين اجاهم"
"شو رات السفجات صحب يكل لا تغني تكي انتقاءي وهب الدجسات البحدين لاقيت عري وان الضي في حتي ولهانت كومان ابا شجف وانيس بنحم عظت لتان فكو اعجب ما لا وجمع فاني مسا ل"
"مرنق اره فقا لها هذاق بنا في شعود الناءبان لان كانا كانا امان والطغدة فيذحلا ولقد فيفود العام عذاق عا؁ر لعين في"
"ضخ ارك عينا تينلا لان يزريني وانلا تصمرك فيان نقتينا هذاقة غربكني سباد يلمادء يفرفش صوتي سكوي دمنا اسوقي مسل حدينيمه لا تعني تكي ان عمدا ورعدا وسنيك نان ركونة تحسوع طءينا الاحزين كمان الان يكل لا ت

In [95]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [72]:
sess = gpt2.start_tf_sess()

In [80]:
import tensorflow as tf

In [97]:
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-100


## Generate text

* length: The length of the generated text in terms of tokens. It determines how many tokens the generated output will contain. Note that the actual length of the output may vary depending on the model's behavior.

* temperature: A parameter that controls the randomness of the generated text. Higher values (e.g., above 1.0) result in more random and diverse output, while lower values (e.g., below 1.0) make the output more focused and deterministic.

* prefix: A starting prompt or seed text from which the generation begins. The generated text will continue from the given prefix.

* nsamples: The number of independent samples to generate. Each sample is a separate generated output. Setting a higher value for nsamples will result in multiple generated texts.

* batch_size: The number of samples to generate in parallel. Specifying a higher batch_size can improve generation speed but requires more computational resources.

In [98]:
text = gpt2.generate(sess,
                     length=100,
                     run_name="run1",
                     temperature=0.7,
                     prefix="في القدس"
                     )

��ي القدسين استروص دوي عين الحين الذي فيها تناري ويها القدسين بلاء ورفك القدسين استروص ويها القدسين بلاء ورفك الق


In [143]:
text2 = gpt2.generate(sess,
                     length=100,
                     run_name="run1",
                     temperature=0.7,
                     prefix="بادني ابتسام منك"
                     )

بادني ابتسام منكر من نعيميه علي يا شكل السخل من زاعتي ويحر من مشاء ساءعاني نصيخ فوقي بيتهك المشوءي ولا فرده المشاءي ور


## Evaluation
The evaluation of the Arabic Poems Generator can be performed based on several aspects:

Model Performance: The generator's performance can be evaluated by examining the quality and coherence of the generated poems. Metrics such as perplexity and BLEU score can provide quantitative insights into the model's performance.

Human Interpretability: The generated poems can be evaluated for their human interpretability. Human evaluators can assess the poetic elements, aesthetics, and overall coherence of the generated poems. Subjective feedback and opinions can help gauge the success of the generator in producing meaningful and engaging poetry.

### I used Similarity between the generated and the actual texts to evaluate the performance. 

In [144]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [146]:
text1 = " بادني ابتسام منك تحيا القراءح وتقوي من الجسم الضعيف الجوارح ومن ذا الذي يقضي حقوقك كلها ومن ذا الذي يرضي سوي من تسامح وقد تقبل العذر الخفي تكرما فما بال عذري واقفا وهو واضح وان محالا اذ بك العيش ان اري وجسمك معتل وجسمي صالح وما كان ترك الشعر الا لانه تقصر عن وصف الامير المداءح"
text2 = "بادني ابتسام منكر من نعيميه علي يا شكل السخل من زاعتي ويحر من مشاء ساءعاني نصيخ فوقي بيتهك المشوءي ولا فرده المشاءي ور"

# Create a TF-IDF vectorizer object
vectorizer = TfidfVectorizer()

# Compute TF-IDF vectors for the texts
tfidf_vectors = vectorizer.fit_transform([text1, text2])

# Compute cosine similarity between the TF-IDF vectors
similarity = cosine_similarity(tfidf_vectors[0], tfidf_vectors[1])[0, 0]

print("Similarity:", similarity)


Similarity: 0.1067785321901297


In [148]:
text1 = eval_data[4000]
text1

'حقاني حتي الكفر ومريض بالخير يدعو دوما لايمان ويظل يرد ايات القران ويحث الناس علي التقوي و الاحسان لكن في السر يسب النسوان الاءي لا يهبطن اليه لاخماد البركان فهو الاقوي والاولي بالرقة والاثر في تبريد الحرقة 196 عبدالخالق سبحان اله والحمد له والشكر له واله اله علي عبدالخالق مفتون بالخمرة والنسوة والغلمان وافلام الجنس وصنع الخير لكل العالم مخلوق لحسنات ومغروم بالعشرة عبد الحسنات وطلاب النشوة لا يعشق الا الفقراء ولا يعبد الا الشهوة والافلام الزرقاء وترتيب السهرة والسهرة تعقبها السهرة والفكرة تخلقها السكرة والانس الذاهب في الحسرة والعطف علي الايتام يزيل الاثام وباب الرحمن فسيح الارجاء وسبحان اله علي عبد الشهوات نجي الدعوات نظيف الفرج عفيف الشرج نقي الحفرة الاصلح والانكح في الحضرة الاتقي والانقي في الصلوات وفي طلب السترة ويتوب الرب علي الكفرة ويتوب الرب علي عبدالخالق يا توبته يا خيبته يا توبتها يا خيبتها كانت في دار ابيها مثل الوردة لا تطبخ لا تكنس لا تمسح لا تحمل غير حقيبتها تلهو تفسح تضحك تمرح تحلي بالالوان وتسرح حتي جاء القدر الماحق وتزوجها عبدالخالق يا خيبته يا خيبتها صارت مثل الالة لا

In [149]:
text2 = gpt2.generate(sess,
                     length=100,
                     run_name="run1",
                     temperature=0.7,
                     prefix="حقاني حتي الكفر"
                     )

��قاني حتي الكفري يضيف الرغاب تكتق الشاطين في الجوين عصرية الوجة جوينة وطويبة الظلالة العامين القلاء عصرية التراقير من ت


In [163]:
text2 = "��قاني حتي الكفري يضيف الرغاب تكتق الشاطين في الجوين عصرية الوجة جوينة وطويبة الظلالة العامين القلاء عصرية التراقير من ت"


# Create a TF-IDF vectorizer object
vectorizer = TfidfVectorizer()

# Compute TF-IDF vectors for the texts
tfidf_vectors = vectorizer.fit_transform([text1, text2])

# Compute cosine similarity between the TF-IDF vectors
similarity = cosine_similarity(tfidf_vectors[0], tfidf_vectors[1])[0, 0]

print("Similarity:", similarity)

Similarity: 0.03876735761320467


In [164]:
text1 = eval_data[4010]
text1

'1 هل ترك الصحراء ماء حدودها يروي جباه التاءهين ام انها ان ذاب ذاك الغصن في اكوابها جاءت ترد اغنيات المتعبين لا لن تحين في الرمل اغنية البكاء وصمتها ينثال في اغفاءة المطر المسافر كل حين يا ارضنا الاولي ويا نبضاتها الاءي توارثن القصاءد في الصدي اين الوجوه? تصاعدت فيها حكايات السنين وتجعدت احلامها فينا كليل باغتوا فيه اشتعالات المواءد واختفي فيهم يرد اغنيات المتعبين 2 هل انت حين يلامس النخل ارتعاشات النهار هل انت حين يهب في الاحداق فجر لغبار تستوطن الوجع المعبا في بنادقنا وتنهض لحصار? قل لايادي انها في الجيب لن تبقي ولن يجتثها لحماسنا صوت انفجار 3 ان هياوا لارض باب وتنفسوا تحت التراب حملوا السيوف علي مواجعهم وعادوا بالغياب سيحاصرون بصمتهم ويحملون الي الذهاب'

In [166]:
text2 = gpt2.generate(sess,
                     length=100,
                     run_name="run1",
                     temperature=0.7,
                     prefix="هل ترك الصحراء ماء"
                     )

هل ترك الصحراء ماء نخص غشار جنونها تحابها فاء بعدة بالمحم في جءيلا والشكون وقتلة وجهوني لمدني واشكوني فوق الصحرا�


In [168]:
text2 = "هل ترك الصحراء ماء نخص غشار جنونها تحابها فاء بعدة بالمحم في جءيلا والشكون وقتلة وجهوني لمدني واشكوني فوق الصحرا�"

# Create a TF-IDF vectorizer object
vectorizer = TfidfVectorizer()

# Compute TF-IDF vectors for the texts
tfidf_vectors = vectorizer.fit_transform([text1, text2])

# Compute cosine similarity between the TF-IDF vectors
similarity = cosine_similarity(tfidf_vectors[0], tfidf_vectors[1])[0, 0]

print("Similarity:", similarity)

Similarity: 0.12753975902817416
